# CIA Factbook Data Analysis with SQL

The [CIA Factbook](https://www.cia.gov/the-world-factbook/), or World Factbook, is an annual publication by the US Central Intelligence Agency, offering essential insights by summarizing global country and regional data, encompassing demographics and geography, among other facets.

![img](cia_factbook.jpg)


In this project, we'll deploy SQL within Jupyter Notebook to scrutinize the data in the SQLite factbook.db database. The objective is to pinpoint global leaders and laggards using demographic measures such as population, mortality, and fertility.

## Connecting to the Database
<hr>

In [17]:
%%capture
%load_ext sql
%sql sqlite:///Desktop/factbook.db

In [18]:
%%sql
SELECT name
FROM sqlite_master
WHERE type='table';

 * sqlite:///Desktop/factbook.db
Done.


name
sqlite_sequence
facts


## Overview of the Data
<hr>

We'll begin by exploring the data.

In [19]:
%%sql
SELECT *
FROM facts
LIMIT 10;

 * sqlite:///Desktop/factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46
6,ac,Antigua and Barbuda,442,442,0,92436,1.24,15.85,5.69,2.21
7,ar,Argentina,2780400,2736690,43710,43431886,0.93,16.64,7.33,0.0
8,am,Armenia,29743,28203,1540,3056382,0.15,13.61,9.34,5.8
9,as,Australia,7741220,7682300,58920,22751014,1.07,12.15,7.14,5.65
10,au,Austria,83871,82445,1426,8665550,0.55,9.41,9.42,5.56


The 'facts' table consists of **11 columns** with descriptive names:
 - `code` — The country's [internet code](https://www.cia.gov/the-world-factbook/field/internet-country-code/).
 - `name` — Name of the country.
 - `area` — The country's total area (both land and water).
 - `area_land` — The country's land area in square kilometers.
 - `area_water` — The country's water area in square kilometers.
 - `population` — Whole number of people or inhabitants in the country.
 - `population_growth` — The country's population growth as a percentage.
 - `birth_rate` — The number of births per year per 1,000 inhabitants.
 - `death_rate` — The number of deaths per year per 1,000 inhabitants.
 - `migration_rate` — The difference between the number of persons entering and leaving the country during the year per 1,000 persons.

Let's calculate the total number of records:

In [10]:
%%sql
SELECT COUNT(*) AS num_records,
       COUNT(DISTINCT name) AS unique_countries
  FROM facts;

 * sqlite:///Desktop/factbook.db
Done.


num_records,unique_countries
261,261


We know there are only 195 countries in the world right now. The reason behind the discrepancy is how the CIA organizes its data. The CIA's World Factbook contains 267 entities, categorized as follows:

1. **Independent countries**: This category has 195 entries, representing sovereign states with distinct territories.
2. **Others**: There are 2 entities here - Taiwan and the European Union.
3. **Dependencies and Areas of Special Sovereignty**: This group has 58 entries linked to other countries.
4. **Miscellaneous**: Antarctica and disputed areas make up 6 entities.
5. **Other entities**: Comprising the world and its 5 oceans.

To explore this in our data, we can check for these categories.

In [13]:
%%sql
-- check for some instances of "non-country" data
SELECT *
  FROM facts
 WHERE name LIKE '%union' 
    OR name LIKE '%ocean'
    OR name LIKE '%island'
    OR name = 'World';

 * sqlite:///Desktop/factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
197,ee,European Union,4324782,None,None,513949445,0.25,10.2,10.2,2.5
199,kt,Christmas Island,135,135,0,1530,1.11,None,None,None
203,nf,Norfolk Island,36,36,0,2210,0.01,None,None,None
208,ip,Clipperton Island,6,6,0,None,None,None,None,None
222,bv,Bouvet Island,49,49,0,None,None,None,None,None
244,bq,Navassa Island,5,5,0,None,None,None,None,None
248,wq,Wake Island,6,6,0,None,None,None,None,None
256,xq,Arctic Ocean,None,None,None,None,None,None,None,None
257,zh,Atlantic Ocean,None,None,None,None,None,None,None,None
258,xo,Indian Ocean,None,None,None,None,None,None,None,None


Given the existence of these instances, it's crucial to be cautious to prevent them from affecting our analysis. Let's  now move forward with our analysis while keeping these factors in mind.

## Exploring Total Population
<hr>

%%sql 
SELECT MIN(population) AS min_population,
       CAST(AVG(population) AS Integer) AS avg_population,
       MAX(population) AS max_pop
  FROM facts;

Some entities have population values of zero, while another entity is recorded with a population exceeding 7 billion. This appears unlikely given the current global population of around 7.96 billion (Worldometer, Aug 2022). Investigating these anomalies is essential.

In [15]:
%%sql 
-- Investigate the unusual countries/entities
SELECT name, population
  FROM facts
 WHERE population = (SELECT MIN(population) FROM facts) 
    OR population = (SELECT MAX(population) FROM facts);

 * sqlite:///Desktop/factbook.db
Done.


name,population
Antarctica,0
World,7256490011


As seen before, entries like 'World' and 'Antarctica' contribute to these unusual population values. For instance, Antarctica's population consists only of research staff at research stations. Furthermore, the population value of 7,256,490,011 is interesting. It's possible that the CIA collected this data between 2014 and 2015, reflecting global population trends. 

![World Population by Year](world_population.jpg)

***Image source***: [Worldometer](https://www.worldometers.info/world-population/world-population-by-year/).

In [16]:
%%sql 
SELECT MIN(population) AS min_pop,
       CAST(AVG(population) AS Integer) AS avg_pop,
       MAX(population) AS max_pop
  FROM facts
 WHERE population != 0 
   AND name != 'World';

 * sqlite:///Desktop/factbook.db
Done.


min_pop,avg_pop,max_pop
48,32377011,1367485388


The lowest population is 48, while the highest exceeds 1.3 billion. On average, the population hovers around 32 million residents. With this information, we can proceed to pinpoint the most and least populated countries or entities.

## Finding Densely Populated Countries
<hr>


![Population Density](https://images.nationalgeographic.org/image/upload/v1652341489/EducationHub/photos/population-density.png)


In [21]:
%%sql 
SELECT name AS top_5_populated,
       population
  FROM facts
 WHERE population <> 0 
   AND name <> 'World'
 ORDER BY 2 DESC
 LIMIT 5;

 * sqlite:///Desktop/factbook.db
Done.


top_5_populated,population
China,1367485388
India,1251695584
European Union,513949445
United States,321368864
Indonesia,255993674


The European Union is not a country; it is better to filter out the EU. Next, we'll calculate the proportion of the world's population that each country holds.

In [22]:
%%sql 
SELECT name AS "5 Most Populated Countries",
       population,
       ROUND(
              CAST(population AS Float) / 
              CAST((SELECT MAX(population) FROM facts) AS Float)
             , 3) AS world_proportion
  FROM facts
 WHERE population <> 0 
   AND name NOT IN ('World', 'European Union')
 ORDER BY 2 DESC
 LIMIT 5;

 * sqlite:///Desktop/factbook.db
Done.


5 Most Populated Countries,population,world_proportion
China,1367485388,0.188
India,1251695584,0.172
United States,321368864,0.044
Indonesia,255993674,0.035
Brazil,204259812,0.028


 - China leads with 1.37 billion people, followed by India with 1.26 billion. China and India together contribute to 36% of the world's population. 

 - The United States, Indonesia, and Brazil follow with 11%, adding up to 46% of the global population.

## Exploring Population Density
<hr>

![Population Density](population_density.jpg)

[National Geographic](https://education.nationalgeographic.org/resource/population-density/) simplifies this concept: population density refers to the number of individuals within a species in a particular geographic area. Utilizing population density data helps measure demographic details and understand connections with ecosystems, human well-being, and infrastructure.

Our dataset offers variables that allow us to estimate population density as outlined below:

$$
Population\ density = \frac{Total\ population}{Land\ area\ (km^2)}
$$

Before calculating population densities for each country, it's essential to remove any instances where population density or land area might be zero or null.

In [24]:
%%sql
SELECT COUNT(*)
  FROM facts
 WHERE population = 0
     OR population IS NULL
     OR area_land = 0
     OR area_land IS NULL;

 * sqlite:///Desktop/factbook.db
Done.


COUNT(*)
29


In [25]:
%%sql
SELECT name, area_land, population,
       CAST(
            CAST(population AS Float) / 
              CAST(area_land AS Float) 
           AS Integer) AS population_density
  FROM facts
-- Filter out countries with zero or missing population/land area
 WHERE name NOT IN (SELECT name
                      FROM facts
                     WHERE population = 0
                         OR population IS NULL
                         OR area_land = 0
                         OR area_land IS NULL
                   )
 ORDER BY 4 DESC
 LIMIT 10;

 * sqlite:///Desktop/factbook.db
Done.


name,area_land,population,population_density
Macau,28,592731,21168
Monaco,2,30535,15267
Singapore,687,5674472,8259
Hong Kong,1073,7141106,6655
Gaza Strip,360,1869055,5191
Gibraltar,6,29258,4876
Bahrain,760,1346613,1771
Maldives,298,393253,1319
Malta,316,413965,1310
Bermuda,54,70196,1299


It's worth highlighting that these places are notably compact. The collective land area of the top five is under 3,000 km². Additionally, many of these locations are urbanized hubs of affluence and finance, leaning towards cityscapes and skyscrapers rather than open landscapes or extensive wilderness ([see here](https://worldpopulationreview.com/country-rankings/countries-by-density)).

To potentially mitigate this bias, we might consider redefining what qualifies as a densely populated country. In our context, we will define a densely populated area as one with less land area and a higher population count compared to other regions.

$$
For\ a\ densely\ populated\ country\ or\ territory:\\
Total\ population > Average\ world\ population\\
AND\ Land\ area < World\ average\ land\ area
$$

Let's use this rule to identify the most densely populated countries:

In [26]:
%%sql
-- Identify countries with zero or missing population/land area
WITH t1 AS (SELECT name
              FROM facts
             WHERE population = 0
                 OR population IS NULL
                 OR area_land = 0
                 OR area_land IS NULL
           ),

-- Calculate population density for countries without missing data  
     t2 AS (SELECT name, area_land, population,
                   CAST(
                        CAST(population AS Float) / 
                          CAST(area_land AS Float) 
                      AS Integer) AS population_density
              FROM facts
             WHERE name NOT IN t1)
    
-- Isolate countries with high population and low land area
SELECT *
  FROM t2
 WHERE area_land < (SELECT AVG(area_land) FROM t2)
     AND population > (SELECT AVG(population) FROM t2)
 ORDER BY population_density DESC
 LIMIT 10;

 * sqlite:///Desktop/factbook.db
Done.


name,area_land,population,population_density
Bangladesh,130170,168957745,1297
"Korea, South",96920,49115196,506
Japan,364485,126919659,348
Philippines,298170,100998376,338
Vietnam,310070,94348835,304
United Kingdom,241930,64088222,264
Germany,348672,80854408,231
Nepal,143351,31551305,220
Italy,294140,61855120,210
Uganda,197100,37101745,188


Bangladesh takes the lead with a combination of high population and limited land area, resulting in a population density of 1,297 people per square kilometer. Notably, Bangladesh's population density is more than double that of the second-ranked South Korea. Japan, the Philippines, and Vietnam come after, continuing the list.

## Mortality and Fertility

The fluctuation in the global population hinges on two key metrics: births and deaths. In 2021, the worldwide crude death rate stood at 7.64 deaths per thousand people, while the crude birth rate reached 17.76 births per thousand. This ratio implies approximately 2.3 births for each death.

Now, let's extract the corresponding data from our dataset:

In [27]:
%%sql 
SELECT birth_rate, death_rate,
       ROUND((birth_rate / death_rate), 2) AS birth_to_death_ratio
  FROM facts
 WHERE name = 'World';

 * sqlite:///Desktop/factbook.db
Done.


birth_rate,death_rate,birth_to_death_ratio
18.6,7.8,2.38


Our data uncovers a comparable trend, indicating approximately 2.4 births for every recorded death.

### Countries with the Highest Birth Rates

In [28]:
%%sql 
SELECT name, birth_rate
  FROM facts
 WHERE name != 'World'
 ORDER BY 2 DESC
 LIMIT 10;

 * sqlite:///Desktop/factbook.db
Done.


name,birth_rate
Niger,45.45
Mali,44.99
Uganda,43.79
Zambia,42.13
Burkina Faso,42.03
Burundi,42.01
Malawi,41.56
Somalia,40.45
Angola,38.78
Mozambique,38.58


Niger has the highest birth rate in the world at 45.45 average annual births per 1,000 people per year. Mali (44.99), Uganda (43.79), Zambia (42.13), and Burkina Faso (42.03) follow closely behind. All countries in the top-10 list belong to Africa.

Significant population growth within a nation is often linked to factors such as a struggling economy, limited awareness and preventive measures, or challenging living conditions. The absence of accessible birth control and contraception in developing countries further amplifies higher fertility rates. Statistical data has indicated that the fertility rate per woman in low-income countries is twice the global average. Consequently, this phenomenon is reflected in the fact that all ten of the countries with the highest birth rates are situated in Africa, as depicted earlier.

### Countries with the Lowest Birth Rates

In [29]:
%%sql 
SELECT name, birth_rate
  FROM facts
 WHERE name != 'World'
-- Filter out entries without birth rates
     AND birth_rate IS NOT NULL
 ORDER BY 2 ASC
 LIMIT 10;

 * sqlite:///Desktop/factbook.db
Done.


name,birth_rate
Monaco,6.65
Saint Pierre and Miquelon,7.42
Japan,7.93
Andorra,8.13
"Korea, South",8.19
Singapore,8.27
Slovenia,8.42
Germany,8.47
Taiwan,8.47
San Marino,8.63


Conversely, the CIA's calculations reveal that Monaco holds the title for the world's lowest birth rate, standing at an average of 6.65 annual births per 1,000 individuals. The inclusion of Saint Pierre on the list is easily understandable given its small population size. Japan (7.93), Andorra (8.13), and South Korea (8.19) follow closely, securing the 3rd to 5th positions.

This [Wikipedia article](https://en.wikipedia.org/wiki/Total_fertility_rate) delves into the factors behind the low birth rates observed in Asian regions such as Japan, South Korea, Taiwan, and Singapore. It identifies aging populations and socio-economic challenges as the primary drivers of these outcomes.

### Countries with the Highest Death Rates

In [30]:
%%sql 
SELECT name, death_rate
  FROM facts
 WHERE name != 'World'
 ORDER BY 2 DESC
 LIMIT 10;

 * sqlite:///Desktop/factbook.db
Done.


name,death_rate
Lesotho,14.89
Ukraine,14.46
Bulgaria,14.44
Guinea-Bissau,14.33
Latvia,14.31
Chad,14.28
Lithuania,14.27
Namibia,13.91
Afghanistan,13.89
Central African Republic,13.8


Ranking in the 1st and 4th positions are African nations Lesotho (with a death rate of 14.89 deaths per 100 people) and Guinea-Bissau (with a rate of 14.33). Reports from the CDC and the Borgen Project indicate that infections and poverty stand as the primary causes of death in these regions.

Meanwhile, the European countries Ukraine (14.46), Bulgaria (14.44), and Latvia (14.3) secure the 2nd, 3rd, and 5th positions, respectively. In contrast to the pattern observed in African countries, studies unveil that non-infectious diseases like circulatory, digestive, respiratory diseases, and cancers are the leading causes of death in these European territories. Poor healthcare administration is identified as a contributing factor to death in Ukraine and Latvia.

As global efforts to combat infectious diseases improve, sources have documented declining death rates in Lesotho and Guinea-Bissau. In contrast, Latvia, Ukraine, and Bulgaria are grappling with increasing death rates.

### Countries with the Lowest Death Rates

In [31]:
%%sql 
SELECT name, death_rate
  FROM facts
 WHERE name != 'World'
-- Filter out entries without birth rates
     AND death_rate IS NOT NULL
 ORDER BY 2
 LIMIT 10;

 * sqlite:///Desktop/factbook.db
Done.


name,death_rate
Qatar,1.53
United Arab Emirates,1.97
Kuwait,2.18
Bahrain,2.69
Gaza Strip,3.04
Turks and Caicos Islands,3.1
Saudi Arabia,3.33
Oman,3.36
Singapore,3.43
West Bank,3.5


Qatar boasts the world's lowest mortality rate, standing at 1.53 deaths per 1,000 individuals. Following closely are the UAE (1.97), Kuwait (2.8), and Bahrain (2.69). These nations have garnered recognition for their advanced healthcare systems, state-of-the-art medical facilities, and exceptional ability to provide top-tier patient care.

Another intriguing observation emerges: Six out of the ten countries listed (Saudi Arabia, Kuwait, the United Arab Emirates, Qatar, Bahrain, and Oman) are members of the Gulf Cooperation Council (GCC). In fact, the entire roster of GCC member states is represented on this list. The GCC forms a political and economic alliance encompassing the six Middle Eastern nations mentioned earlier.

## Countries with the Fastest Declining Population
<HR>

To pinpoint the swiftest declining populations, we target nations that satisfy the following criteria:

1. A birth-to-death ratio below one, indicating a lack of natural population increase.
2. Minimal or negligible population growth, characterized by growth rates of less than 1%.
3. Extremely low levels of immigration, with migration rates below one.

In [32]:
%%sql
SELECT name,
       ROUND((birth_rate / death_rate), 2) AS birth_death_ratio, 
       population_growth,
       migration_rate
  FROM facts
 WHERE birth_death_ratio < 1
     AND population_growth < 1
     AND migration_rate < 1
 ORDER BY 4, 2, 3;

 * sqlite:///Desktop/factbook.db
Done.


name,birth_death_ratio,population_growth,migration_rate
Serbia,0.66,0.46,0.0
Japan,0.83,0.16,0.0
Romania,0.77,0.3,0.24
Bulgaria,0.62,0.58,0.29
Slovenia,0.74,0.26,0.37
Bosnia and Herzegovina,0.91,0.13,0.38
Poland,0.96,0.09,0.46
Belarus,0.8,0.2,0.7


The primary factor behind the sharp population decline in Eastern Europe is attributed to birth rates. Research from the Peterson Institute for International Economics (PIIE) indicates that the region's birth rates have decreased significantly since the dissolution of the Soviet Union.

However, the decline in birth rates alone is insufficient to counterbalance the combined effects of mortality and emigration. The Eastern European nations experienced multiple waves of emigration subsequent to the enlargement of the European Union's (EU) borders in 2004 and 2007. According to the PIIE, as of 2016, approximately 6.3 million Eastern European individuals had migrated to other EU member states.

Since 1970, there has been a persistent decline in birth rates within Japan. The fertility rate of Japan has been graphed alongside diverse predictions made over the years. Despite these predictions projecting an upward trajectory, the actual rates have consistently decreased with each passing period.

## Conclusion
<hr>

Throughout this project, we've utilized the CIA World Factbook data to pinpoint the leading and lagging regions globally. Our analysis hinges on demographic indicators such as population, mortality, and fertility.

>- China, India, the United States, Indonesia, and Brazil, contributing to 46% of the global population, are the most populous countries. Notably, sparsely inhabited areas are mainly islands and dependencies.
>- In terms of population density, Macau, Monaco, Singapore, and Hong Kong emerge as more densely populated due to their limited land area. In contrast, Bangladesh, South Korea, and Japan exhibit high population density considering their larger land areas.
>- Fertility rates peak in Africa while being lowest in Europe and Asia. Some African (Lesotho, Guinea-Bissau) and Eastern European (Bulgaria, Lithuania, Serbia) nations report higher death rates. The Gulf Cooperation Council nations in the Middle East, conversely, have the lowest mortality rates.
>- Eastern and South-Eastern Europe are undergoing significant population declines, attributed to reduced birth rates, elevated mortality, and emigration. Even though Japan has optimistic projections, it's experiencing a population decrease.